# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
nltk.download('averaged_perceptron_tagger')
from sklearn import svm
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\quart\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\quart\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\quart\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine) 
X = df['message'] + " " + df['genre']
#X = df[['message','genre']]
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
lr = LogisticRegression(max_iter=500, random_state=42)
pipeline =  Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('clf', MultiOutputClassifier(lr)),
    ])

In [33]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', TfidfVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x0000016DB44DF380>)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(max_iter=500,
                                                      random_state=42)))],
 'verbose': False,
 'vect': TfidfVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x0000016DB44DF380>),
 'clf': MultiOutputClassifier(estimator=LogisticRegression(max_iter=500,
                                                    random_state=42)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accent

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=0) 

In [35]:
pipeline.fit(X_train, y_train)

(19681,)
(19681, 35)


Pipeline(steps=[('vect',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x0000016DB44DF380>)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(max_iter=500,
                                                                    random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [5]:
def display_results(y_test, y_pred):
    
    target_names = y_test.columns
    
    print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))



In [37]:
# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      5050
               request       0.47      0.12      0.19      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.47      0.27      0.34      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.22      0.00      0.01       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [38]:
parameters = {
        'clf__estimator__max_iter': [500, 750],
        'vect__ngram_range': ((1, 1), (1, 2)),
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [39]:
cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

display_results(y_test, y_pred)

print("\nBest Parameters:", cv.best_params_) #27m 50s


                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      5050
               request       0.45      0.13      0.20      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.47      0.24      0.32      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.33      0.00      0.00       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [40]:
rf = RandomForestClassifier()

pipeline_rf =  Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('clf', MultiOutputClassifier(rf)),
    ])

pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)
display_results(y_test, y_pred_rf) #27min 38secs

                        precision    recall  f1-score   support

               related       0.77      0.98      0.87      5050
               request       0.34      0.03      0.06      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.46      0.18      0.26      2691
          medical_help       0.07      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.22      0.00      0.01       414
                  food       0.12      0.00      0.01       682
               shelter       0.10      0.01      0.01       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [42]:
kn =KNeighborsClassifier()
pipeline_kn =  Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('clf', MultiOutputClassifier(kn)),
    ])

pipeline_kn.fit(X_train, y_train)
y_pred_kn = pipeline_kn.predict(X_test) #very quick 2m 25s
display_results(y_test, y_pred_kn)

                        precision    recall  f1-score   support

               related       0.78      0.98      0.86      5050
               request       0.30      0.25      0.27      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.44      0.39      0.41      2691
          medical_help       0.40      0.00      0.01       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.19      0.01      0.01       682
               shelter       0.07      0.02      0.03       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [43]:
mlp =MLPClassifier(early_stopping=True, max_iter=100)
pipeline_mlp =  Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('clf', MultiOutputClassifier(mlp)),
    ])

pipeline_mlp.fit(X_train, y_train)
y_pred_mlp = pipeline_mlp.predict(X_test)
display_results(y_test, y_pred_mlp) #147m 49secs

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5050
               request       0.48      0.13      0.21      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.62      0.00      0.01      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.00      0.00      0.00       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [44]:
rc = RidgeClassifier()
pipeline_rc =  Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize, token_pattern=None)),
        ('clf', MultiOutputClassifier(rc)),
    ])

pipeline_rc.fit(X_train, y_train)
y_pred_rc = pipeline_rc.predict(X_test)
display_results(y_test, y_pred_rc) #1m 24s

                        precision    recall  f1-score   support

               related       0.78      0.95      0.86      5050
               request       0.43      0.15      0.23      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.46      0.36      0.41      2691
          medical_help       0.50      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.24      0.01      0.02       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [45]:
pipeline_rf.get_params()

{'memory': None,
 'steps': [('vect', TfidfVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x0000016DB44DF380>)),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': TfidfVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x0000016DB44DF380>),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__use_idf': True,
 'vect__v

In [ ]:
parameters_rf = {
        'clf__estimator__criterion': ['gini', 'log_loss'],
        'vect__ngram_range': [(1, 2)],
    }

cv_rf = GridSearchCV(pipeline_rf, param_grid=parameters_rf)

In [65]:
cv_rf.fit(X_train, y_train)
y_pred_rf = cv_rf.predict(X_test)
display_results(y_test, y_pred_rf)
print("\nBest Parameters:", cv_rf.best_params_)

c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


                        precision    recall  f1-score   support

               related       0.77      0.98      0.87      5050
               request       0.34      0.02      0.05      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.46      0.23      0.30      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.00      0.00      0.00       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [7]:
#random forest is not a great improvement and takes a long time
parameters = {
        'clf__estimator__max_iter': [500],
        'clf__estimator__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
        'clf__estimator__penalty': ['l2', None],
        'vect__ngram_range': [(1, 2)],

    }

cv_lr = GridSearchCV(pipeline, param_grid=parameters)

In [10]:
cv_lr.fit(X_train, y_train)
y_pred_lr = cv_lr.predict(X_test)
display_results(y_test, y_pred_lr)
print("\nBest Parameters:", cv_lr.best_params_) #105m 33s

c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      5050
               request       0.45      0.17      0.24      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.47      0.25      0.33      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.20      0.00      0.00       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [18]:

pipeline_lr_verb = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),
    
    ('clf', MultiOutputClassifier(LogisticRegression())),
])

pipeline_lr_verb.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x000001F1C9CCBF60>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('starting_verb', StartingVerbExtractor())])),
  ('clf', MultiOutputClassifier(estimator=LogisticRegression()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001F1C9CCBF60>)),
                                                 ('tfidf',
                                                  TfidfTransformer())])),
                                ('starting_ver

In [24]:
parameters = {
        'clf__estimator__max_iter': [500, 750],
        'clf__estimator__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
        'clf__estimator__penalty': ['l2', None],
        'features__text_pipeline__vect__ngram_range': [(1, 2)],

    }


cv_lr_verb = GridSearchCV(pipeline_lr_verb, param_grid=parameters)

In [25]:
cv_lr_verb.fit(X_train, y_train)
y_pred_lr_verb = cv_lr_verb.predict(X_test)
display_results(y_test, y_pred_lr_verb)
print("\nBest Parameters:", cv_lr_verb.best_params_) #346m 21s

c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used sin

                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      5050
               request       0.44      0.17      0.24      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.47      0.25      0.33      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.33      0.00      0.01       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

In [26]:
class Genre(BaseEstimator, TransformerMixin):

    def genre(self, text):
        return text.split()[-1]

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.get_dummies(pd.Series(X).apply(self.genre))
        return pd.DataFrame(X_tagged)

In [27]:
pipeline_lr_verb_genre = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor()),
        ('genre', Genre())
    ])),

    ('clf', MultiOutputClassifier(LogisticRegression())),
])


cv_lr_verb_genre = GridSearchCV(pipeline_lr_verb_genre, param_grid=parameters)

In [28]:
cv_lr_verb_genre.fit(X_train, y_train)
y_pred_lr_verb_genre = cv_lr_verb_genre.predict(X_test)
display_results(y_test, y_pred_lr_verb_genre)
print("\nBest Parameters:", cv_lr_verb_genre.best_params_)

c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\quart\anaconda3\envs\udemy_course\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used sin

                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      5050
               request       0.49      0.12      0.19      1093
                 offer       0.00      0.00      0.00        28
           aid_related       0.46      0.24      0.31      2691
          medical_help       0.00      0.00      0.00       537
      medical_products       0.00      0.00      0.00       329
     search_and_rescue       0.00      0.00      0.00       182
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       207
                 water       0.00      0.00      0.00       414
                  food       0.33      0.00      0.00       682
               shelter       0.00      0.00      0.00       545
              clothing       0.00      0.00      0.00       103
                 money       0.00      0.00      0.00       154
        missing_people       0.00      

### 9. Export your model as a pickle file

In [ ]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(cv_lr_verb_genre, open(filename, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.